## Trace without setting environment variables

In [4]:
import openai
from langsmith import Client, tracing_context, traceable
from langsmith.wrappers import wrap_openai

langsmith_client = Client(
    api_key="",
    api_url="https://api.smith.langchain.com"
)

client = wrap_openai(openai.Client())

@traceable(run_type="tool", name="Retrieve Context")
def my_tool(question: str) -> str:
    return "During this morning's meeting, we solved all world conflict."

@traceable
def chat_pipeline(question: str):
    context = my_tool(question)
    messages = [
        {"role": "system", "content": "You are a helpful assistant. Please respond to the user's request only based on the given context."},
        {"role": "user", "content": f"Question: {question} \n Context: {context}"}
    ]
    chat_completion = client.chat.completions.create(
        model='gpt-4o-mini', messages=messages
    )
    return chat_completion.choices[0].message.content

# Can set to False to disable tracing here without changing code structure
with tracing_context(enabled=True):
    # Use langsmith_extra to pass in a custom client
    chat_pipeline("Can you summarize this morning's meetings?", langsmith_extra={"client": langsmith_client})